# https://wikidocs.net/152606

아래의 실습은 구글의 Colab을 사용한다고 가정하고 작성되었습니다. 다른 환경에서 실습하셔도 상관은 없지만, 형태소 분석기 Mecab은 다른 방법으로 설치하셔야 합니다.

In [ ]:
# 위키피디아로부터 데이터를 파싱하기 위한 파이썬 패키지인 wikiextractor를 설치합니다.
!pip install wikiextractor

In [ ]:
# 위키피디아 데이터를 다운로드 한 후에 전처리에서 사용할 형태소 분석기인 Mecab을 설치합니다.
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
# 위키피디아 덤프(위키피디아 데이터)를 다운로드합니다.
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

In [ ]:
# 위키익스트랙터를 사용하여 위키피디아 덤프를 파싱합니다.
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

In [ ]:
# 현재 경로에 있는 디렉토리와 파일들의 리스트를 받아옵니다.
%ls

In [ ]:
import os
import re
os.listdir('text')

In [ ]:
%ls text/AA

In [ ]:
# 이제 이 6개 AA ~ AF 디렉토리 안의 wiki 숫자 형태의 수많은 파일들을 하나로 통합하는 과정을 진행해야 합니다. 
# AA ~ AF 디렉토리 안의 모든 파일들의 경로를 파이썬의 리스트 형태로 저장합니다.
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            # 재귀 함수
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)
filepaths = list_wiki('text')

In [ ]:
len(filepaths)

In [ ]:
# 총 파일의 개수는 850개입니다. 이제 output_file.txt라는 파일에 850개의 파일을 전부 하나로 합칩니다.

with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

In [ ]:
# 파일을 읽고 10줄만 출력해보겠습니다.

f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

## 2. 형태소 분석 

In [ ]:
from tqdm import tqdm
from konlpy.tag import Mecab 

In [ ]:
mecab = Mecab()

In [ ]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

In [ ]:
lines[:10]

In [ ]:
result = []

for line in tqdm(lines):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result.append(mecab.morphs(line))

In [ ]:
len(result)

### 3. Word2Vec 학습


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)